In [157]:
import pandas as pd
from glob import glob 

In [267]:
#Part 1 cohort selection 

In [158]:
mag_view  = pd.read_csv("/embed/tables/embed_datathon_magview_full.csv",dtype='str') 
mag_view = mag_view[mag_view['asses'].isin(['N','B','A'])].copy() # get the studies of interest 
meta = pd.read_csv("/embed/tables/embed_datathon_metadata_reduced.csv",dtype='str')
meta = meta[meta['acc_anon'].isin(mag_view['acc_anon'].unique())].copy()

In [268]:
meta = meta[meta['FinalImageType']=='2D'] #eliminate c-view images 

In [276]:
#merge the magview and the metadata 
combi = pd.merge(meta,mag_view,left_on=['acc_anon','ImageLateralityFinal'],right_on=['acc_anon','side'])

In [277]:
combi = combi.drop_duplicates(subset=['png_path'])# ensure we  don't have duplicates after merger 

In [278]:
combi = combi[combi['StudyDescription'].str.contains('Screen')]

In [279]:
#split cohorts by asesment of findings 
acc_only = combi.drop_duplicates(subset='acc_anon')

In [273]:
birads0 = acc_only[acc_only['asses']=='A'].sample(5000,random_state=1996).copy()
birads1 = acc_only[acc_only['asses']=='B'].sample(2000,random_state=1996).copy()
birads2 = acc_only[acc_only['asses']=='N'].sample(2000,random_state=1996).copy()

In [280]:
final_acc =  pd.concat([birads0,birads1,birads2])

In [275]:
final_combi = combi[combi['acc_anon'].isin(final_acc['acc_anon'])].copy()

In [69]:
final_combi['view_combination'] = final_combi['ImageLateralityFinal'] + final_combi['ViewPosition']

In [70]:
final_combi = final_combi.drop_duplicates(subset=['acc_anon','view_combination'])

In [73]:
final_combi.shape

(20251, 141)

In [78]:
final_combi.to_csv('./final_infer_meta.csv',index=False)

In [166]:
final_combi = pd.read_csv('./final_infer_meta.csv',dtype='str')

In [167]:
#had to restart testing so this basically makes another csv for the remaining cases. 
infered = pd.concat([pd.read_csv(e,dtype='str') for e in glob('/shared/covnext_run2/*.csv')])

In [281]:
missing = final_combi[~final_combi['png_path'].isin(infered['png_path'])]

In [172]:
missing.to_csv('./final_infer_meta_run2.csv',index=False)

In [170]:
final_combi.shape

(20251, 141)

In [171]:
infered.shape

(13001, 142)

In [2]:
!ls /shared/covnext_run2

predicted_batch_0.csv  predicted_batch_2.csv
predicted_batch_1.csv  predicted_batch_3.csv


In [118]:
import pandas as pd
from glob import glob 

In [142]:
# viewing model predictions 

sample = pd.concat([pd.read_csv(e) for e in  glob("/shared/covnext_run2/*.csv") ])

In [143]:
sample = sample[sample['asses']=='A']

In [144]:
sample.shape

(6931, 142)

In [145]:
#sample = sample[sample['path_severity'].isin([0,1])]

In [146]:
sample.shape

(6931, 142)

In [147]:
sample = sample[sample['side']==sample['ImageLateralityFinal']]
sample['cancer_pred'] = (sample['score']>=0.31).astype(int)

In [148]:
sample = sample.sort_values(by=['acc_anon','cancer_pred'],ascending=False)

In [149]:
sample = sample.drop_duplicates(subset='acc_anon')

In [173]:
sample['side'].value_counts()

side
R    1547
L    1484
Name: count, dtype: int64

In [150]:
sample.shape

(3031, 143)

In [153]:
for  n, sub_d in sample.groupby(by='ETHNICITY_DESC'): 
    print(f" NAME is :{n} with {sub_d.shape[0]})") 
    print(sub_d['cancer_pred'].value_counts(normalize=True))

 NAME is :African American or Black with 1352)
cancer_pred
0    0.914941
1    0.085059
Name: proportion, dtype: float64
 NAME is :American Indian or Alaskan Native with 14)
cancer_pred
0    0.928571
1    0.071429
Name: proportion, dtype: float64
 NAME is :Asian with 182)
cancer_pred
0    0.956044
1    0.043956
Name: proportion, dtype: float64
 NAME is :Caucasian or White with 1173)
cancer_pred
0    0.920716
1    0.079284
Name: proportion, dtype: float64
 NAME is :Multiple with 16)
cancer_pred
0    0.875
1    0.125
Name: proportion, dtype: float64
 NAME is :Native Hawaiian or Other Pacific Islander with 40)
cancer_pred
0    0.925
1    0.075
Name: proportion, dtype: float64
 NAME is :Unknown, Unavailable or Unreported with 237)
cancer_pred
0    0.949367
1    0.050633
Name: proportion, dtype: float64


In [174]:
!ls /shared

covnext_output	predictions_left_black.csv   predictions_right_white.csv
covnext_run2	predictions_left_white.csv
covnext_run3	predictions_right_black.csv


In [257]:
out=  pd.concat([pd.read_csv(e) for e in glob('/shared/predictions*') ]).reset_index()

/tmp/ipykernel_96708/2379895787.py:1: DtypeWarning: Columns (66,82,114) have mixed types. Specify dtype option on import or set low_memory=False.
  out=  pd.concat([pd.read_csv(e) for e in glob('/shared/predictions*') ]).reset_index()
/tmp/ipykernel_96708/2379895787.py:1: DtypeWarning: Columns (57,66,76,77,78,79,83,84) have mixed types. Specify dtype option on import or set low_memory=False.
  out=  pd.concat([pd.read_csv(e) for e in glob('/shared/predictions*') ]).reset_index()
/tmp/ipykernel_96708/2379895787.py:1: DtypeWarning: Columns (37,66,84,86,112) have mixed types. Specify dtype option on import or set low_memory=False.
  out=  pd.concat([pd.read_csv(e) for e in glob('/shared/predictions*') ]).reset_index()


In [258]:
from tableone import TableOne

In [265]:

columns = ['ETHNICITY_DESC','path_severity_bin','asses']

In [266]:
TableOne(out.drop_duplicates(subset=['acc_anon']),columns=columns,categorical=columns)

Missing      Overall
n                                                                  7728
ETHNICITY_DESC, n (%)    African American or Black       0  3869 (50.1)
                         Caucasian or White                 3859 (49.9)
path_severity_bin, n (%) 0                               0  7603 (98.4)
                         1                                    125 (1.6)
asses, n (%)             A                               0  4103 (53.1)
                         B                                  1815 (23.5)
                         N                                  1810 (23.4)

In [ ]:
TableOne(out.drop_duplicates(subset=['acc_anon']),columns=columns,categorical=columns)

In [239]:
import seaborn as sns

In [231]:
import numpy as np 

#out = out[np.logical_and(out['total_L_find']==1.0,out['tissueden']==3.0)]

In [282]:
sub_d = out[['y_hat','ETHNICITY_DESC','path_severity_bin']].copy()

In [283]:
sub_d.columns = ['score','RACE','label_value']

In [284]:
from aequitas.preprocessing import preprocess_input_df
from aequitas.group import Group
from aequitas.plotting import Plot
from aequitas.bias import Bias
df, _ = preprocess_input_df(sub_d)

In [285]:
g = Group()
x_tab,_ = g.get_crosstabs(df)
aqp = Plot() 
b= Bias() 
bdf = b.get_disparity_predefined_groups(x_tab,original_df=sub_d,ref_groups_dict={'RACE':'Caucasian or White'},mask_significance=False)
sub= bdf[['attribute_value','tpr','fpr','fnr','tnr','tpr_disparity','fpr_disparity','fnr_disparity','tnr_disparity']].copy()
sub.round(3)

get_disparity_predefined_group()


,attribute_value,tpr,fpr,fnr,tnr,tpr_disparity,fpr_disparity,fnr_disparity,tnr_disparity
0,African American or Black,0.344,0.038,0.656,0.962,0.779,1.232,1.175,0.993
1,Caucasian or White,0.441,0.031,0.559,0.969,1.000,1.000,1.000,1.000


In [286]:
sub.round(3)

,attribute_value,tpr,fpr,fnr,tnr,tpr_disparity,fpr_disparity,fnr_disparity,tnr_disparity
0,African American or Black,0.344,0.038,0.656,0.962,0.779,1.232,1.175,0.993
1,Caucasian or White,0.441,0.031,0.559,0.969,1.000,1.000,1.000,1.000
